In [1]:
import sys

sys.path.insert(0, "..")
import warnings

from gpbp.layers import AdmArea

warnings.filterwarnings(action="ignore")

/Users/pkollenda/Library/Caches/pypoetry/virtualenvs/gpbp-Gm2-lxOw-py3.10/lib/python3.10/site-packages/geopandas/_compat.py:154: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  set_use_pygeos()


## Defining the Administrative Area

In [2]:
adm_area = AdmArea(country="Timor-Leste", level=0)
adm_area.get_adm_area("Timor-Leste")
adm_area.get_facilities(method="osm", tags={"building": "hospital"})
adm_area.get_population(method="world_pop")
adm_area.compute_potential_fac(spacing=0.05)

Retrieving data for Timor-Leste of granularity level 0
Extracting geometry for Timor-Leste
Retrieving {'building': 'hospital'} for Timor-Leste area
Data downloaded
Converting raster file to dataframe


## Retrieving the road network

In [3]:
adm_area.get_road_network("driving")

#### How is the road network used?
In distance.py the calculate_isopolygons_graph method uses the road network and the Lat, Lon coordinates of the facilities. 


In [14]:
X = adm_area.fac_gdf.drop(columns=["geometry"]).longitude.to_list()
Y = adm_area.fac_gdf.drop(columns=["geometry"]).latitude.to_list()
G = adm_area.road_network

In [16]:
import osmnx as ox

rode_nodes = ox.distance.nearest_nodes(G, X, Y)
rode_nodes

[5790100400,
 7709476609,
 10569143716,
 9222819647,
 9222819671,
 1421854313,
 1421854313,
 1421854313,
 2788685044,
 1783791890,
 910528138,
 11092709098,
 5225733008,
 3475912707,
 494069204,
 5593426832,
 1783195431,
 1783195431,
 1793520802,
 1793520802,
 1793520802,
 1793520825,
 1793520825]

In [20]:
import networkx as nx

for road_node in rode_nodes:
    print(nx.ego_graph(G, road_node, radius=1000, distance="length"))

MultiDiGraph with 14 nodes and 26 edges
MultiDiGraph with 54 nodes and 138 edges
MultiDiGraph with 1 nodes and 0 edges
MultiDiGraph with 72 nodes and 154 edges
MultiDiGraph with 71 nodes and 152 edges
MultiDiGraph with 36 nodes and 74 edges
MultiDiGraph with 36 nodes and 74 edges
MultiDiGraph with 36 nodes and 74 edges
MultiDiGraph with 182 nodes and 479 edges
MultiDiGraph with 8 nodes and 16 edges
MultiDiGraph with 11 nodes and 24 edges
MultiDiGraph with 48 nodes and 136 edges
MultiDiGraph with 3 nodes and 4 edges
MultiDiGraph with 12 nodes and 30 edges
MultiDiGraph with 1 nodes and 0 edges
MultiDiGraph with 5 nodes and 8 edges
MultiDiGraph with 4 nodes and 6 edges
MultiDiGraph with 4 nodes and 6 edges
MultiDiGraph with 79 nodes and 192 edges
MultiDiGraph with 79 nodes and 192 edges
MultiDiGraph with 79 nodes and 192 edges
MultiDiGraph with 78 nodes and 190 edges
MultiDiGraph with 78 nodes and 190 edges


## Prepare optimization data

In [5]:
MAPBOX_API_TOKEN = None  # fill out with your own access token for mapbox strategy
DISTANCE_TYPE = "length"
pop_count, current, potential = adm_area.prepare_optimization_data(
    distance_type="length",  # "length" or "time" // what unit to measure the distance in.
    distance_values=[2000],  # which isopolygons to draw (here 2 km)
    mode_of_transport="driving",  # travel type (driving, walking, cycling)
    strategy="osm",  # "strategy", OSM (requires adm_area.road_network) or "mapbox" (requires MAPBOX_API_TOKEN)
    population_resolution=5,
    mapbox_access_token=MAPBOX_API_TOKEN,
)

/Users/pkollenda/Data & Development/CorrelAid/Public-Infrastructure-Service-Access/examples/../gpbp/distance.py:252: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  serve_df = pd.DataFrame(index=fac_gdf["ID"].values, data=serve_dict).applymap(
/Users/pkollenda/Data & Development/CorrelAid/Public-Infrastructure-Service-Access/examples/../gpbp/distance.py:252: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  serve_df = pd.DataFrame(index=fac_gdf["ID"].values, data=serve_dict).applymap(


In [ ]:
# Returns dataframe with facilities and households "served" by the facility.
current["length"]